### Analyse Spatiale et Prédective

# Application d'outils d'analyse spatiale sur lames minces grâce à Python

Choisir une image à traiter dans le même dossier que le notebook Jupyter avec nom_de_image.format :

### Image à traiter (exemple : lame2.png) : 

In [ ]:
a = str(input())
h = './images/'
file = h + a

In [ ]:
#Modules necessaires:
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
from skimage import data, io, img_as_ubyte

image_originale = io.imread(file) # Image originale 
image = rgb2gray(io.imread(file)) # Image en Noir et Blanc

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15, 3.5))

ax[0].imshow(image_originale) # image de base en niveaux de gris
ax[0].set_title('Image originale')
ax[0].axis('off')

ax[1].hist(image.ravel()*255, bins=255) 
ax[1].set_title('Histogramme niveaux de gris')
ax[1].set_xlabel('Niveaux de gris')
ax[1].set_ylabel('Nombre de pixels')
plt.show()

#### Nombre de classe souhaité ? (entre 2 et 3)

In [ ]:
nombre_phases_minerales = int(input()) #nombre de classe desiré
if nombre_phases_minerales > 3 or nombre_phases_minerales < 2 : #si 1 ou plus de 4 minéraux (=classe)
    print ('Erreur dans le choix du nombre de phases minérales (doit être entre 2 et 3)')
    nombre_phases_minerales = int(input())

In [ ]:
import numpy as np
import skimage
from skimage.filters import threshold_multiotsu

thresholds = threshold_multiotsu(image, classes=nombre_phases_minerales) #multi Otsu
regions = np.digitize(image, bins=thresholds)
output = img_as_ubyte(regions) 

### Résultat seuillage Multi-Otsu :

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(20, 5))
ax[0].imshow(image_originale) # image de base en niveaux de gris
ax[0].set_title('Image originale')
ax[0].axis('off')

ax[1].hist(image.ravel()*255, bins=255) # Histogramme + multi Otsu
ax[1].set_title('Histogramme des niveaux de gris seuillé')
for thresh in thresholds:
    ax[1].axvline(thresh*255, color='r')
ax[1].set_xlabel('Niveaux de gris')
ax[1].set_ylabel('Nombre de pixels')

ax[2].imshow(regions,cmap='jet') # Resultat multi Otsu
ax[2].set_title('Resultat classification Multi-Otsu ')
ax[2].axis('off')

In [ ]:
import cv2 
plt.imsave('./Resultat_Otsu.png', output)
Otsu = cv2.imread('./Resultat_Otsu.png')

In [ ]:
from skimage import color, morphology
import time
import matplotlib.cbook as cbook
from scipy import misc
import matplotlib.image as img
from PIL import Image
from math import * #new
from scipy.stats import gaussian_kde #new
import math
import seaborn as sns #new
from scipy import stats #new
import ipywidgets as widgets
from ipywidgets import interact

### Séparation des phases : 

In [ ]:
hsv = cv2.cvtColor(Otsu, cv2.COLOR_BGR2HSV)
if nombre_phases_minerales >= 2: #si 2 minéraux
 lower_rangej = np.array([17,0,0],dtype=np.uint8) #JAUNE
 upper_rangej = np.array([37,255,255],dtype=np.uint8) #JAUNE
 maskj = cv2.inRange(hsv, lower_rangej, upper_rangej)
 outputj = img_as_ubyte(maskj)
 plt.imsave('./mineral1.png', outputj)

 img = Image.open('./mineral1.png')
 thresh = 200
 fn = lambda x : 255 if x < thresh else 0
 mineral1gray = img.convert('L').point(fn, mode='1')
 mineral1gray.save('./mineral1.png')

 lower_rangev = np.array([130,0,0],dtype=np.uint8) #VIOLET
 upper_rangev = np.array([160,255,255],dtype=np.uint8) #VIOLET
 maskv = cv2.inRange(hsv, lower_rangev, upper_rangev)
 outputv = img_as_ubyte(maskv)
 plt.imsave('./mineral2.png', outputv)

 img = Image.open('./mineral2.png')
 thresh = 200
 fn = lambda x : 255 if x < thresh else 0
 mineral2gray = img.convert('L').point(fn, mode='1')
 mineral2gray.save('./mineral2.png')

 # Image en sortie: resultat
 if nombre_phases_minerales ==2:
    fig, ax = plt.subplots(nrows=1, ncols=nombre_phases_minerales, figsize=(10, 3.5))

    ax[0].imshow(maskj, cmap='OrRd')
    ax[0].set_title('mineral 1')
    ax[0].axis('off')

    ax[1].imshow(maskv, cmap='GnBu')
    ax[1].set_title('mineral 2')
    ax[1].axis('off')

if nombre_phases_minerales == 3: #si 3 minéraux

 lower_rangec = np.array([79,0,0]) #CYAN
 upper_rangec = np.array([99,255,255]) #CYAN
 maskc = cv2.inRange(hsv, lower_rangec, upper_rangec)
 outputc = img_as_ubyte(maskc)
 plt.imsave('./mineral3.png', outputc)

 img = Image.open('./mineral3.png')
 thresh = 200
 fn = lambda x : 255 if x < thresh else 0
 mineral3gray = img.convert('L').point(fn, mode='1')
 mineral3gray.save('./mineral3.png')

 # Image en sortie: resultat
 fig, ax = plt.subplots(ncols=nombre_phases_minerales, figsize=(40, 10))

 ax[0].imshow(maskj,cmap='OrRd')
 ax[0].set_title('phase 1', fontsize = 25)
 ax[0].axis('off')

 ax[1].imshow(maskv,cmap='GnBu')
 ax[1].set_title('phase 2', fontsize = 25)
 ax[1].axis('off')

 ax[2].imshow(maskc,cmap='BuGn')
 ax[2].set_title('phase 3', fontsize = 25)
 ax[2].axis('off')


plt.subplots_adjust()
plt.show()

cv2.waitKey(0)
cv2.destroyAllWindows()

### Pourcentages des phases : 

In [ ]:
if nombre_phases_minerales == 2: #Si 2 minéraux

 m = io.imread('./mineral1.png')
 a = np.array([m.shape]) #extraction taille image mineral1gray

 hauteur = a[0][0]#largeur image mineral1gray
 largeur = a[0][1] #longueur image mineral1gray

 nbpxm=((hauteur*largeur*255)-np.sum(m))/255
 
 pcm1=round((nbpxm*100)/(largeur*hauteur),1) #pourcentage mineral1
 print(pcm1,'% pour la phase 1') 

 pcm2=round(100-pcm1,1) #pourcentage mineral2
 print(pcm2,'% pour la phase 2') 


if nombre_phases_minerales == 3: #Si 3 minéraux 

 m = io.imread('./mineral1.png')
 a = np.array([m.shape]) #extraction taille image mineral1gray

 hauteur = a[0][0]#largeur image mineral1gray
 largeur = a[0][1] #longueur image mineral1gray

 nbpxm=((hauteur*largeur*255)-np.sum(m))/255 #nb de pixel

 pcm1=round((nbpxm*100)/(largeur*hauteur),1) #pourcentage mineral1
 print(pcm1,'% pour la phase 1') 

 m = io.imread('./mineral2.png')
 a = np.array([m.shape]) #extraction taille image mineral2gray

 hauteur = a[0][0]#largeur image mineral2gray
 largeur = a[0][1] #longueur image mineral2gray

 nbpxm=((hauteur*largeur*255)-np.sum(m))/255

 pcm2=round((nbpxm*100)/(largeur*hauteur),1) #pourcentage mineral2
 print(pcm2,'% pour la phase 2') 


 pcm3=round(100-(pcm1+pcm2),1) #pourcentage mineral3
 print(pcm3,'% pour la phase 3') 

#### Quelle phase traiter? (1 à 3)

In [ ]:
numeromineral = int(input()) #choix du mineral à traiter
if numeromineral == 1:
    nomimage = './mineral1.png'
elif numeromineral == 2:
    nomimage = './mineral2.png'
elif numeromineral == 3:
    nomimage ='./mineral3.png'

#### Puissance du filtre anti-poussière? (entre 0 et 8)

In [ ]:
filtre = int(input()) #puissance du filtre

In [ ]:
#1°)Première transformation inverse noir et blanc 
img = Image.open(nomimage)
thresh = 200
fn = lambda x : 255 if x < thresh else 0
mineralgray = img.convert('L').point(fn, mode='1')
mineralgray.save(nomimage)

#2°)application du filtre
image1 = cv2.imread(nomimage)
image = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
selem =  morphology.disk(filtre) 
res = morphology.white_tophat(image, selem)

#3°)affichage 3 images (original,filtre, orignal-filtre)
fig, ax = plt.subplots(ncols=2, figsize=(20, 8))
ax[0].set_title('Avant filtre',fontsize = 15)
ax[0].imshow(image, cmap='binary')
ax[0].axis('off')
ax[1].set_title('Après filtre',fontsize =15)
ax[1].imshow(image - res, cmap='binary')
ax[1].axis('off')
plt.imsave('./mineralFiltre.png', image-res, cmap='gray')
plt.show()

#transformation inverse noir et blanc (remise a la normale)
img = Image.open('./mineralFiltre.png')
img2 = Image.open(nomimage)
thresh = 200
fn = lambda x : 255 if x < thresh else 0
mineralgray = img.convert('L').point(fn, mode='1')
mineralgrayy = img2.convert('L').point(fn, mode='1')
mineralgray.save('./mineralFiltre.png')
mineralgrayy.save(nomimage)

### Detection des centres: 

In [ ]:
imageori= file #image originale à changer


#1°)transformation en matrice
imagenom='./mineralFiltre.png' #image à traiter
image = cv2.imread(imagenom)
imageorii=cv2.imread(imageori)
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
ret,thresh = cv2.threshold(gray_image,127,255,0)
taille_img = np.array([image.shape]) 
hauteur=taille_img[0][0]
largeur=taille_img[0][1]
imagevierge=np.zeros((hauteur,largeur))+255 #image de même taille mais vierge (blanche)
imagevierge1=np.zeros((hauteur,largeur))+255

#2°) cherche coutours 
contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

#3°) cherche centres
for c in contours:
   M = cv2.moments(c)
   if M["m00"] != 0:
     cX = int(M["m10"] / M["m00"])
     cY = int(M["m01"] / M["m00"])
     

     if gray_image[cY,cX] == 0:
         cv2.circle(image, (cX, cY), 3, (0, 0, 255), 2)
         cv2.circle(imageorii, (cX, cY), 3, (0, 0, 255), 2)
         cv2.circle(imagevierge, (cX, cY), 3, (0, 0, 0), 2)
         cv2.circle(imagevierge1, (cX, cY), 0, (0, 0, 0), -1)


#5°)sauvegarde
#plt.imsave(r'C:\Users\pauly\Desktop\projetAScode\imageAS\mineralCentreFiltre.png', image)                                      pas obligatoire
plt.imsave('./mineralCentreHough.png',imagevierge)                            
plt.imsave('./mineralCentreUnique.png', imagevierge1)

imge = Image.open('./mineralCentreUnique.png') #imageCentreUnique à inverser les couleurs
thresh = 200
fn = lambda x : 255 if x > thresh else 0
mineral1centre3= imge.convert('L').point(fn, mode='1')
mineral1centre3.save('./mineralCentreUnique.png')
imge = Image.open('./mineralCentreHough.png') #imageCentreUnique à inverser les couleurs
thresh = 200
fn = lambda x : 255 if x > thresh else 0
mineral1centre3= imge.convert('L').point(fn, mode='1')
mineral1centre3.save('./mineralCentreHough.png')
     
#4°)affichage
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(40, 15))

ax[0].imshow(image)
ax[0].set_title('Centres sur la phase étudiée', fontsize = 30)
ax[0].axis('off')

ax[1].imshow(imagevierge,'Greys_r')
ax[1].set_title('Centres seuls', fontsize = 30)
ax[1].axis('off')

ax[2].imshow(imageorii)
ax[2].set_title('Centres sur image originale', fontsize = 30)
ax[2].axis('off')


In [ ]:
from ipywidgets import IntProgress
from IPython.display import display
import time
img = './mineralCentreUnique.png' #image à traiter

#1°) transformation matrice
imgmat = io.imread(img) 
taillemat = np.array([imgmat.shape]) 
hauteur = taillemat[0][0]
largeur = taillemat[0][1] 

centreimg = np.zeros((1,2)) 
centreimg[0,0]=ceil(hauteur/2) 
centreimg[0,1]=ceil(largeur/2) 
imgvide = np.zeros((hauteur,largeur))+255 

#2°) recuperation des centres

count=0
for ha in range (hauteur): #boucle récupération XY des centres
    for la in range (largeur):
        if imgmat[ha,la] == 0:
            count=count+1
            
if numeromineral ==1:
    taillec=(pcm1/100*(hauteur*largeur))/count
elif numeromineral ==2:
    taillec=(pcm2/100*(hauteur*largeur))/count
elif numeromineral ==3:
    taillec=(pcm3/100*(hauteur*largeur))/count
print(count, 'cristaux detectés')
print(largeur*hauteur,'pixels^2 est la superficie de l image')
print(round(taillec), ' pixels^2 est la taille moyenne de la phase étudiée')

## Méthode de Fry : 

Veuillez patientez le temps de chargement (10 à 30 s) 

In [ ]:

count1=count
matriceXcYc = np.zeros((2,count1)) #matrice coordonnées Xc et Yc des centres

colX=0
colY=0
for ha in range (hauteur):
    for la in range (largeur):
        if imgmat[ha,la] == 0:
            count=count+1
            matriceXcYc[colY,colX]=ha
            matriceXcYc[colY+1,colX]=la
            colY=0
            colX=colX+1
colYy=0
cbcentre=0

#3°) décallage+impression
for colXx in range (count1):
    minx=(centreimg[0,0]-matriceXcYc[colYy,colXx])#décalage X
    miny=(centreimg[0,1]-matriceXcYc[colYy+1,colXx])#décalage Y
    matriceXcYcd=matriceXcYc #matrice après décalage
    colY=0
 
    for colX in range (count1): #matrice décalée
        matriceXcYcd[colY,colX]=matriceXcYc[colY,colX]+(minx)
        matriceXcYcd[colY+1,colX]=matriceXcYc[colY+1,colX]+(miny)
        colY=0

    colY=0
    colX=0
    for colX in range (count1):
        if matriceXcYcd[colY][colX] < hauteur and matriceXcYcd[colY][colX] > 0 and matriceXcYcd[colY+1][colX] < largeur and matriceXcYcd[colY+1][colX] > 0:
            imgvide[int(matriceXcYcd[colY][colX]),int(matriceXcYcd[colY+1][colX])]=0
            cbcentre=cbcentre+1
        else:
            colY=0
    
#4°)enregistrement
plt.imsave('./mineralFry.png', imgvide) #fry enregistrement

img = Image.open('./mineralFry.png') #image 64 couleurs a transformer en 1 couleur
thresh = 200
fn = lambda x : 255 if x > thresh else 0
mineral1grayy = img.convert('L').point(fn, mode='1')
mineral1grayy.save('./mineralFry.png') 
mineral1grayyy = io.imread('./mineralFry.png') #trnasformation matrice

#5°)densité de Kernel
hh=0

for ha in range (hauteur):
    for la in range (largeur):
        if mineral1grayyy[ha,la] == 0:
            hh=hh+1

matriceXcentre = np.zeros((1,hh)) 
matriceYcentre= np.zeros((1,hh)) 

colXY=0
for ha in range (hauteur):
    for la in range (largeur):
        if mineral1grayyy[ha,la] == 0:
            matriceXcentre[0,colXY]=ha
            matriceYcentre[0,colXY]=la
            colXY=colXY+1

x = matriceXcentre
y = matriceYcentre
m1, m2 = y, -x
xmin = m1.min()
xmax = m1.max()
ymin = m2.min()
ymax = m2.max()

X, Y = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
positions = np.vstack([X.ravel(), Y.ravel()])
values = np.vstack([m1, m2])
kernel = stats.gaussian_kde(values)
Z = np.reshape(kernel(positions).T, X.shape)

#3°)affichage 3 images (original,filtre, orignal-filtre)
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 8))
ax[0].set_title('Analyse Fry', fontsize=15)
ax[0].imshow(mineral1grayy, cmap='binary')
ax[0].axis('off')

ax[1].set_title('Densité de Kernel', fontsize= 15)
ax[1].imshow(np.rot90(Z), cmap=plt.cm.gist_ncar,
           extent=[xmin, xmax, ymin, ymax])
#ax.plot(m1, m2, 'k.', markersize=1) #afficher centre sur graph
ax[1].set_xlim([xmin, xmax])
ax[1].set_ylim([ymin, ymax])
plt.show()

### Analyse de distance au voisinage (DNN) : 

In [ ]:
from numpy.random import randn
from matplotlib.ticker import FuncFormatter

img = './mineralCentreUnique.png' #image à traiter

#1°) transformation matrice
imgmat = io.imread(img) 
taillemat = np.array([imgmat.shape]) 
hauteur = taillemat[0][0]
largeur = taillemat[0][1] 

centreimg = np.zeros((1,2)) 
centreimg[0,0]=ceil(hauteur/2) 
centreimg[0,1]=ceil(largeur/2) 
imgvide = np.zeros((hauteur,largeur))+255 

#2°) recuperation des centres

count=0
for ha in range (hauteur): #boucle récupération XY des centres
    for la in range (largeur):
        if imgmat[ha,la] == 0:
            count=count+1


count1=count
matriceXcYc = np.zeros((2,count1)) #matrice coordonnées Xc et Yc des centres
matriceDistance = np.zeros((1,count1)) #distance du voisin le plus proche de chaque centre
matriceDistancefinale = np.zeros((1,count1)) #
colX=0
colY=0
for ha in range (hauteur):
    for la in range (largeur):
        if imgmat[ha,la] == 0:
            count=count+1
            matriceXcYc[colY,colX]=ha
            matriceXcYc[colY+1,colX]=la
            colY=0
            colX=colX+1


u=0
U2=0
for f in range (count1):
    for d in range (count1):
        matriceDistance[0][0+U2]=sqrt((matriceXcYc[0,0+u]-matriceXcYc[0][U2])**2+(matriceXcYc[1,0+u]-matriceXcYc[1][U2])**2)
        U2=U2+1
    matriceDistancefinale[0][0+u]=matriceDistance[matriceDistance>0].min()
    u=u+1
    U2=0


max=np.max(matriceDistancefinale)
x = list(matriceDistancefinale)
r0=round(matriceDistancefinale.mean(),2)
print (r0,' pixels est la distance moyenne (r0) des voisins les plus proches (DNN)')


p=count1/(largeur*hauteur)
print(round(p,6), 'est la densité de centre par pixel (p)')

re=1/(sqrt(p))
print(round(re,2),'pixels est la distance du voisin le plus proche (rE) si la distribution est aléatoire')

R=r0/re

if R <0.9:
    print(round(R,2), 'est l indice de dispertion (R) donc la répartition est de type "cluster" ')
elif R >1.1:
    print(round(R,2), 'est l indice de dispertion (R) donc la répartition est de type "dispertion" ')
else:
    print(round(R,2), 'est l indice de dispertion (R) donc la répartition est de type "aléatoire" ')

    
plt.hist(x, range = (0, max), bins = 12, color = 'grey',
            edgecolor = 'k')
plt.xlabel('distance du voisin le plus proche (pixel)')
plt.ylabel('fréquence')
plt.title('Analyse de distance au voisinage (DNN)')
plt.axvline(x=r0, color = 'r')
plt.axvline(x=re, color = 'b')
plt.legend([plt.axvline(x=r0, color = 'r'),plt.axvline(x=re, color = 'b')], ['moyenne observée','moyenne attendue'])
plt.show()


### Méthode de Hough : 

In [ ]:
import numpy as np
from skimage.transform import hough_line, hough_line_peaks
from skimage.feature import canny
from skimage.draw import line
from skimage import data

import matplotlib.pyplot as plt
from matplotlib import cm
file= './mineralCentreHough.png'
imgmat=io.imread(file)
hauteur,largeur = imgmat.shape 
#Changement de couleurs: 
for ha in range (hauteur):
    for la in range (largeur):
        if imgmat[ha,la] == 0:
            imgmat[ha,la]=1
        else:
            imgmat[ha,la]=0 
image = io.imread(file)

# Classic straight-line Hough transform
# Set a precision of 0.5 degree.
tested_angles = np.linspace(-np.pi / 2, np.pi / 2, 360, endpoint=True)
h, theta, d = hough_line(imgmat, theta=tested_angles)
d=d/10
# Generating figure 1
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
ax = axes.ravel()

ax[0].imshow(image, cmap=cm.gray)
ax[0].set_title('Centres seuls')
ax[0].set_axis_off()

bounds = [np.rad2deg(theta[0] ),
          np.rad2deg(theta[-1]),
          d[-1], d[0]]
ax[1].imshow(np.log(1 + h), extent=bounds, cmap=cm.gray, aspect=1 / 1.5)
ax[1].set_title('Transformée de Hough')
ax[1].set_xlabel('Angles (degrés)')
ax[1].set_ylabel('Distance (pixels*10^1)')
ax[1].axis('image')



plt.tight_layout()

plt.show()